In [2]:
import sys
sys.path.append("../")
import geopandas as gpd
import pandas as pd
from info import info
import os

In [3]:
elections = info["WI_20"]["elections"]

In [18]:
def initialize_gdf():
    gdf = gpd.read_file("../shapes/WI_vtd20/")

    folder = "/Users/gabe/Dropbox/PlanAnalysis/proposed_plans/Wisconsin"
    plans = []
    for level in ["congress", "state_senate", "state_house"]:
        name = "CD" if level == "congress" else "SD" if level == "state_senate" else "HD"
        for file in os.listdir(f"{folder}/{level}"):
            if "csv" in file:
                if "Enacted" in file:
                    plan_name = f"{file[:-4]}-{name}"
                else:
                    plan_name = f"{file[:-4]}"
                print(plan_name)
                plans.append(plan_name)
                assignment = pd.read_csv(f"{folder}/{level}/{file}").astype(str)
                assignment.columns = ["GEOID20", plan_name]
                gdf = gdf.merge(assignment, on="GEOID20")
    return gdf, plans

In [19]:
gdf, plans = initialize_gdf()

Enacted-CD
PMC-CD
Enacted-SD
PMC-SD
Enacted-HD
PMC-HD


In [20]:
plans

['Enacted-CD', 'PMC-CD', 'Enacted-SD', 'PMC-SD', 'Enacted-HD', 'PMC-HD']

In [21]:
elections["LG18P"]

{'POP_COL': 'VAP',
 'candidates': {'LG18P_BARN': {'name': 'Mandela Barnes',
   'race': 'B',
   'won': True},
  'LG18P_KOBE': {'name': 'Kurt Kober', 'race': 'W', 'won': False}}}

In [22]:
def summarize_plan(gdf, plan):
    gdf[plan] = gdf[plan].astype(int)
    gdf = gdf.groupby(plan).sum()
    gdf["POCVAP20_pct"] = 1 - (gdf["WVAP20"] / gdf["VAP20"])
    gdf["BVAP20_pct"] = gdf["BVAP20"] / gdf["VAP20"]
    gdf["HVAP20_pct"] = gdf["HVAP20"] / gdf["VAP20"]
    gdf["AMINVAP20_pct"] = gdf["AMINVAP20"] / gdf["VAP20"]
    elections = info["WI_20"]["elections"]
    cols = ["POCVAP20_pct", "BVAP20_pct", "HVAP20_pct", "AMINVAP20_pct"]
    for election in ["LG18P", "GOV18P", "SOS18P"]:
        candidates = elections[election]["candidates"].keys()
        tot_vote = gdf[candidates].sum(axis=1)
        for candidate in candidates:
            col = f"{election}_{elections[election]['candidates'][candidate]['name'].split(' ')[-1]}"
            gdf[col] = gdf[candidate] / tot_vote
            cols.append(col)
    return gdf[cols]

In [23]:
for plan in plans:
    summary = summarize_plan(gdf, plan)
    summary.to_csv(f"~/Dropbox/gabe-moon/WI_primary_evals_102821/{plan}.csv")